In [ ]:
import sys
sys.path.append('../')

In [ ]:
from datetime import datetime as dt
import os
import pandas as pd

from mibi_bin_tools import bin_files
from toffy import qc_comp
from ark.utils import io_utils

### Define file parameters

* `base_dir`: this is the top-level directory to store the qc metrics from different runs
* `bin_file_dir`: define the path to your bin file directory
* `panel_path`: define the path to the CSV defining your panel parameters
* `qc_dir`: define the path to write the QC metrics to

In [ ]:
base_dir = 'C:\\Users\\Customer.ION\\Documents\\qc_metrics'
bin_file_path = os.path.join(base_dir, 'tiny_bin_files', '2022-01-22_adam_control_moly_2700V')
panel_path = os.path.join(base_dir, 'sample_panel_2022-01-22_adam_control_moly_2700V.csv')
qc_dir = os.path.join(base_dir, '2022-01-22_adam_control_moly_2700V_metrics')

# make qc_dir if it doesn't already exist
if not os.path.exists(qc_dir):
    os.mkdir(qc_dir)

### Create QC metric tables

Set the following parameters for Gaussian blurring prior to running QC metric analysis:

* `gaussian_blur`: set to `True` if a Gaussian blur is desired
* `blur_factor`: define the standard deviation of the Gaussian kernel. Higher values are more aggressive in smoothing signal. Ignored if `gaussian_blur` is `False`.

In [ ]:
gaussian_blur = True
blur_factor = 1

In [ ]:
# run QC metric extraction, saves to specified qc_dir
qc_comp.compute_qc_metrics(
    bin_file_path,
    panel_path,
    qc_dir,
    gaussian_blur=gaussian_blur,
    blur_factor=blur_factor
)

### Visualize QC metrics

Note: the x-axis (FOV labels) of each graph is ordered in the FOV acquisition order

In [ ]:
# read the data in
df_nonzero_mean = pd.read_csv(os.path.join(qc_dir, 'nonzero_mean_stats.csv'))
df_total_intensity = pd.read_csv(os.path.join(qc_dir, 'total_intensity_stats.csv'))
df_99_9_intensity = pd.read_csv(os.path.join(qc_dir, 'percentile_99_9_stats.csv'))

# melt the DataFrames for visualization
df_nonzero_mean_melt = pd.melt(df_nonzero_mean, id_vars=['fov'], value_vars=channels, var_name='channel', value_name='Non-zero mean intensity')
df_total_intensity_melt = pd.melt(df_total_intensity, id_vars=['fov'], value_vars=channels, var_name='channel', value_name='Total intensity')
df_99_9_intensity_melt = pd.melt(df_99_9_intensity, id_vars=['fov'], value_vars=channels, var_name='channel', value_name='99.9% intensity value')

In [ ]:
# visualize the non-zero mean intensity stats
qc_comp.visualize_qc_metrics(
    df_nonzero_mean_melt,
    'Non-zero mean intensity',
    axes_size=16,
    wrap=6,
    dpi=None,
    save_dir=run_folder_path
)

In [ ]:
# visualize the total intensity stats
qc_comp.visualize_qc_metrics(
    df_total_intensity_melt,
    'Total intensity',
    axes_size=16,
    wrap=6,
    dpi=None,
    save_dir=run_folder_path
)

In [ ]:
# visualize the 99.9% intensity value stats
qc_comp.visualize_qc_metrics(
    df_99_9_intensity_melt,
    '99.9% intensity value',
    axes_size=16,
    wrap=6,
    dpi=None,
    save_dir=run_folder_path
)